### 初始化任务名字（影响数据读取和保存的路径）等

In [1]:
!echo 'jetson' | sudo -S systemctl restart nvargus-daemon && printf '\n'

[sudo] password for jetson: 


In [2]:
import torchvision.transforms as transforms
from dataset import ImageClassificationDataset

TASK = 'test'
width=224
height=224
rootDir = TASK + '/signal'
imagesDir= rootDir + '/images'

CATEGORIES = ['禁行','红绿灯','只许直行','只许左转','只许右行','禁止直行','禁止左转','禁止右转','人行道','无信号']

DATASETS = ['A', 'B']

TRANSFORMS = transforms.Compose([
    transforms.ColorJitter(0.1, 0.1, 0.1, 0.1),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

datasets = {}
for name in DATASETS:
    datasets[name] = ImageClassificationDataset(imagesDir + '_' + name, CATEGORIES, TRANSFORMS)

###  显示收集的数据

In [3]:
import cv2
import ipywidgets
import traitlets
from IPython.display import display
from jetcam.utils import bgr8_to_jpeg
from jupyter_clickable_image_widget import ClickableImageWidget
from imagelist import ImageList
import threading
import time


# initialize active dataset
dataset = datasets[DATASETS[0]]



from jetcam.csi_camera import CSICamera
# from jetcam.usb_camera import USBCamera

camera = CSICamera(width=224, height=224)
# camera = USBCamera(width=224, height=224)

camera.running = True
# create image preview
# 代码段可能会多次运行，所以每次都要先解绑所有摄像头的绑定
camera.unobserve_all()

# 创建预览图像
camera_widget = ClickableImageWidget(width=camera.width, height=camera.height)
traitlets.dlink((camera, 'value'), (camera_widget, 'value'), transform=bgr8_to_jpeg)



data_collection_widget = ipywidgets.VBox([
    ipywidgets.HBox([camera_widget])
])
display(data_collection_widget)


### 训练

In [4]:
import torch
import torchvision

device = torch.device('cuda')
output_dim = len(dataset.categories)  # x, y coordinate for each category

# ALEXNET
# model = torchvision.models.alexnet(pretrained=True)
# model.classifier[-1] = torch.nn.Linear(4096, output_dim)

# SQUEEZENET 
model = torchvision.models.squeezenet1_1(pretrained=True)
model.classifier[1] = torch.nn.Conv2d(512, output_dim, kernel_size=1)
model.num_classes = len(dataset.categories)

# RESNET 18
# model = torchvision.models.resnet18(pretrained=True)
# model.fc = torch.nn.Linear(512, output_dim)

# RESNET 34
# model = torchvision.models.resnet34(pretrained=True)
# model.fc = torch.nn.Linear(512, output_dim)

# DENSENET 121
# model = torchvision.models.densenet121(pretrained=True)
# model.classifier = torch.nn.Linear(model.num_features, output_dim)

model = model.to(device)

model_save_button = ipywidgets.Button(description='保存模型')
model_load_button = ipywidgets.Button(description='加载模型')
model_path_widget = ipywidgets.Text(description='模型文件', value=rootDir+'/model.pth')

def load_model(c):
    model.load_state_dict(torch.load(model_path_widget.value))
model_load_button.on_click(load_model)
    
def save_model(c):
    torch.save(model.state_dict(), model_path_widget.value)
model_save_button.on_click(save_model)
model_save_button.disabled = True
model_widget = ipywidgets.VBox([
    model_path_widget,
    ipywidgets.HBox([model_load_button, model_save_button])
])



### 测试和保存

In [5]:
from utils import preprocess
import torch.nn.functional as F

state_widget = ipywidgets.ToggleButtons(options=['stop', 'live'], description='验证状态', value='stop')
prediction_widget = ipywidgets.Text(description='分析结果')
score_widgets = []
for category in dataset.categories:
    score_widget = ipywidgets.FloatSlider(min=0.0, max=1.0, description=category, orientation='vertical')
    score_widgets.append(score_widget)
    
def live(state_widget, model, prediction_widget):
    global dataset
    while state_widget.value == 'live':
        image = camera.value
        preprocessed = preprocess(image)
        output = model(preprocessed)
        output = F.softmax(output, dim=1).detach().cpu().numpy().flatten()
        category_index = output.argmax()
        prediction_widget.value = dataset.categories[category_index]
        for i, score in enumerate(list(output)):
            score_widgets[i].value = score
            
def start_live(change):
    if change['new'] == 'live':
        execute_thread = threading.Thread(target=live, args=(state_widget, model, prediction_widget))
        execute_thread.start()

state_widget.observe(start_live, names='value')

live_execution_widget = ipywidgets.VBox([
    ipywidgets.HBox(score_widgets),
    prediction_widget,
    state_widget
])





BATCH_SIZE = 8

optimizer = torch.optim.Adam(model.parameters())
# optimizer = torch.optim.SGD(model.parameters(), lr=1e-3, momentum=0.9)


eval_button = ipywidgets.Button(description='验证')

def train_eval(is_training):
    global BATCH_SIZE, LEARNING_RATE, MOMENTUM, model, dataset, optimizer, eval_button, train_button, accuracy_widget, loss_widget, progress_widget, state_widget
    
    try:
        train_loader = torch.utils.data.DataLoader(
            dataset,
            batch_size=BATCH_SIZE,
            shuffle=True
        )

        state_widget.value = 'stop'
        eval_button.disabled = True
        time.sleep(1)

        model = model.eval()
    except e:
        pass
    model = model.eval()

    eval_button.disabled = False
    state_widget.value = 'live'
    
eval_button.on_click(lambda c: train_eval(is_training=False))
    
train_eval_widget = ipywidgets.VBox([
    ipywidgets.HBox([eval_button])
])

all_widget = ipywidgets.VBox([
    ipywidgets.HBox([data_collection_widget, live_execution_widget]), 
    train_eval_widget,
    model_widget
])

display(all_widget)
